In [1]:
# there is a issue on panda after 1.0.0, need 0.25.3
import pandas as pd
pd.__version__

'0.25.3'

In [3]:
#Inorder oo match the review texts with business info, we need to merge the two json files
#Load and Clean yelp_academic_dataset_business.json file

#  local path
business_json_path = 'yelp_academic_dataset_business.json'
business = pd.read_json(business_json_path, lines=True)
print(business.shape)
business.head()

(209393, 14)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [4]:
# There are significant amount of businesses that are not open anymore
business.is_open.value_counts()

1    168903
0     40490
Name: is_open, dtype: int64

In [5]:
# Keep only business that are still open
# Drop columns that may not be relavent
df_business = business[business['is_open']==1].drop(['hours','is_open'], axis=1)
print(df_business.shape)
df_business.head()

(168903, 12)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,..."
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,None,"Pets, Pet Services, Pet Groomers"
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma..."
5,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,"1720 W Elliot Rd, Ste 105",Gilbert,AZ,85233,33.350399,-111.827142,4.5,38,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Automotive, Oil Change Stations, ..."


In [6]:
#Find relevant categories
#One way to split the reviews is by category
# df.explode requires pandas ver 0.25
# Create one row for each series that contain comma-separated items
df_explode = df_business.assign(categories = df_business.categories.str.split(', ')).explode('categories')
df_explode.sample(3)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories
139093,pLBF8QrRkMicJcsjAqW7ag,Walnut Street Tea,115 S Walnut St,Champaign,IL,61820,40.115472,-88.243202,4.0,26,"{'BusinessParking': '{'garage': False, 'street...",Food
84264,7pEOPGaEIr_jgp36zvordA,Country Inn & Suites by Radisson - Mesa,6650 E Superstition Springs,Mesa,AZ,85206,33.385583,-111.686893,3.0,35,"{'RestaurantsPriceRange2': '2', 'WiFi': 'u'fre...",Hotels
74687,XpknXTIXGriiU3Nx9bgqKw,Fairmont Pharmacy,5068 N Central Ave,Phoenix,AZ,85012,33.511150,-112.074130,4.5,20,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",Health & Medical


In [7]:
print('Total number of categories:', len(df_explode.categories.value_counts()))
print('Top 40 categories:')
df_explode.categories.value_counts()[:40]

Total number of categories: 1324
Top 40 categories:


Restaurants                  43965
Shopping                     28480
Food                         24844
Home Services                20653
Health & Medical             17626
Beauty & Spas                17293
Local Services               14319
Automotive                   13149
Nightlife                     9818
Event Planning & Services     9500
Active Life                   8618
Bars                          8573
Fast Food                     7032
Fashion                       6542
Coffee & Tea                  6461
Professional Services         6356
Doctors                       6341
Home & Garden                 6328
Sandwiches                    6094
Hair Salons                   6088
Auto Repair                   5920
Real Estate                   5771
Hotels & Travel               5675
Pizza                         5431
American (Traditional)        5407
Arts & Entertainment          5387
Burgers                       4807
Breakfast & Brunch            4759
Nail Salons         

In [8]:
# Finding categories that contains Dentists
df_explode[df_explode['categories'].str.contains('Fitness & Instruction', case=True, na=False)].categories.value_counts()

Fitness & Instruction    3876
Name: categories, dtype: int64

In [9]:
# Keep only business with categories that are Fitness
business_Fitness = df_business[df_business['categories'].str.contains(
                         'Fitness & Instruction', case=False, na=False)]
print(business_Fitness.shape)
business_Fitness.head()

(3876, 12)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,..."
9,EosRKXIGeSWFYWwpkbhNnA,Xtreme Couture,700 Kipling Avenue Etobicoke,Toronto,ON,M8Z 5G3,43.624539,-79.529108,3.0,16,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Martial Arts, Gyms, Fitness & Instruction, Act..."
19,Vwo64kNYDjKi98gUUv4-iQ,Gateway Fit Body Boot Camp,5229 S Power Rd,Mesa,AZ,85212,33.320723,-111.685869,4.5,16,"{'AcceptsInsurance': 'False', 'GoodForKids': '...","Weight Loss Centers, Fitness & Instruction, Bo..."
56,YFsb1ydMxFLrxtJ3CffVhw,LA Fitness,7640 W Thomas Rd,Phoenix,AZ,85033,33.481551,-112.223225,2.5,32,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Yoga, Active Life, Trainers, Sports Clubs, Fit..."
177,CV6edrz2Lv_kwyAGdswS2A,Fitness Nation,"991 Matheson Boulevard E, Unit 11",Mississauga,ON,L4W 2V3,43.632382,-79.641175,4.5,3,"{'GoodForKids': 'True', 'DogsAllowed': 'False'...","Active Life, Boxing, Gyms, Fitness & Instructi..."


In [10]:
#Loading Massive file as chunks in Pandas
#With a huge file like yelp dataset, loading all the data at once will cost huge RAM.

#Luckily, pandas has an option to load large data by segmenting the file into smaller chunks.

# Set up local path
review_json_path = 'yelp_academic_dataset_review.json'

import pandas as pd

# Set chunk size (smaller if dataset is smaller)
# 2020 Yelp review.json has more than 8 million reviews(rows)
#1 million per time in this dataset will cost more than 6GB of my RAM
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      # identifying the data type of each column can reduce memory usage
                      dtype={'review_id':str,'user_id':str,'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,'funny':int,'cool':int},
                      chunksize=size, encoding="utf8")


In [12]:
#Merge Review.json and Business.json file
## There are multiple chunks to be read
chunk_list = []
for chunk in review:
    # Drop columns that aren't needed
    chunk = chunk.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_Fitness, chunk, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
    
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)
print(df.shape)
df.sample(3)

9171 out of 1,000,000 related reviews
8196 out of 1,000,000 related reviews
8712 out of 1,000,000 related reviews
11199 out of 1,000,000 related reviews
9058 out of 1,000,000 related reviews
8069 out of 1,000,000 related reviews
9468 out of 1,000,000 related reviews
298 out of 1,000,000 related reviews
(64171, 16)


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,user_id,review_stars,text,date
2658,lGbJ55cTkQRdbbZ5yhdMoQ,Reformed Pilates,"15425 N Scottsdale Rd, Ste 250",Scottsdale,AZ,85254,33.626419,-111.924687,4.0,5,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Pilates, Fitness & Instruction, Active Life",mIedYeJMPqKDU9f5ZbUENQ,5,It took me a really long time to find a workou...,2019-12-09 03:42:05
53606,alGc6p7Cjm_OEEw3yfmavQ,Planet Fitness,260 E Lake Mead Pkwy,Henderson,NV,89015,36.041065,-114.977235,4.0,50,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Trainers, Fitness & Instruction, Active Life, ...",-PHC1ulwHkY4LEmMqmFwPg,4,Been twice so far since they opened and I have...,2016-02-22 17:22:32
59688,D9MyvYiPYA8t7gYYk9wY-Q,Orangetheory Fitness - Mountains Edge,"8085 Blue Diamond Rd, Ste 108",Las Vegas,NV,89178,36.019519,-115.267600,5.0,42,"{'ByAppointmentOnly': 'False', 'BusinessParkin...","Boot Camps, Fitness & Instruction, Trainers, G...",8p3K2gLC9QGkoBSZ4ag3ng,5,"Great coaches, great equipment, awesome commun...",2019-03-19 17:57:23


In [13]:
#Review Filter:
#drop review that length are less than 50
df['text_length'] = df['text'].str.len()

In [14]:
df=df.drop(df[df.text_length < 50].index)
df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,user_id,review_stars,text,date,text_length
0,E0TzCCST9IdY6L0vgMdDyA,Ageless Martial Arts,"8525 S Eastern Ave, Ste 100",Las Vegas,NV,89123,36.034695,-115.119496,4.0,25,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Karate, Active Life, Gyms, Martial Arts, Fitne...",6sdvPcRnWEw9MrkIbwhJ9A,1,While Sensei Joe is a genuinely good person an...,2016-09-25 13:33:49,2496
1,E0TzCCST9IdY6L0vgMdDyA,Ageless Martial Arts,"8525 S Eastern Ave, Ste 100",Las Vegas,NV,89123,36.034695,-115.119496,4.0,25,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Karate, Active Life, Gyms, Martial Arts, Fitne...",vkieDbz4qNPYqnOAdiT12g,5,My daughter joined about 5 months ago and she ...,2017-09-14 01:21:04,254
2,E0TzCCST9IdY6L0vgMdDyA,Ageless Martial Arts,"8525 S Eastern Ave, Ste 100",Las Vegas,NV,89123,36.034695,-115.119496,4.0,25,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Karate, Active Life, Gyms, Martial Arts, Fitne...",Bn8i5Wuw5LtS_vlZY3zCUw,1,I had high hopes when I started taking the kic...,2018-06-24 20:01:13,792
3,E0TzCCST9IdY6L0vgMdDyA,Ageless Martial Arts,"8525 S Eastern Ave, Ste 100",Las Vegas,NV,89123,36.034695,-115.119496,4.0,25,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Karate, Active Life, Gyms, Martial Arts, Fitne...",a9st_gGVVwyqaynGPvKTnA,5,"Sensei Joe is a Karate-Geek, and I mean that i...",2016-02-20 06:12:54,592
4,E0TzCCST9IdY6L0vgMdDyA,Ageless Martial Arts,"8525 S Eastern Ave, Ste 100",Las Vegas,NV,89123,36.034695,-115.119496,4.0,25,"{'GoodForKids': 'True', 'BusinessAcceptsCredit...","Karate, Active Life, Gyms, Martial Arts, Fitne...",V8pbzWd_Tii1bwnThrMedA,5,I have been working out on my own for a very l...,2018-05-16 23:48:41,858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64166,yaHyPn3zfLPK46NpvtqlPw,Best Life Nutrition,4158 N Goldwater Blvd,Scottsdale,AZ,85251,33.496624,-111.930538,4.5,34,"{'Caters': 'True', 'RestaurantsTakeOut': 'True...","Juice Bars & Smoothies, Food, Boot Camps, Fitn...",fgQZdw76ck3MC7CjgB0F5A,5,Good little spot. Bowls are huge and tasty. Yo...,2019-08-31 22:41:41,258
64167,yaHyPn3zfLPK46NpvtqlPw,Best Life Nutrition,4158 N Goldwater Blvd,Scottsdale,AZ,85251,33.496624,-111.930538,4.5,34,"{'Caters': 'True', 'RestaurantsTakeOut': 'True...","Juice Bars & Smoothies, Food, Boot Camps, Fitn...",mO6_aOIFs2t1Ie676fPSLw,3,Maybe I'm not the target consumer for this typ...,2019-10-16 00:39:54,1115
64168,qA_A5toPBF5jablugXVnog,Planet Fitness,230 S Decatur,Las Vegas,NV,89107,36.171535,-115.207535,2.5,26,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Trainers, Gyms, Fitness & Instruction, Active ...",FAGCByyGNy5Tqp3RTP6b7g,2,I joined this location in February and was ver...,2016-12-04 12:59:11,905
64169,b-5jNlYZ7i_NcyAIxlrnfQ,Nicole Anne Yoga,,Chandler,AZ,85249,33.223669,-111.821087,5.0,3,"{'AcceptsInsurance': 'False', 'BusinessAccepts...","Yoga, Active Life, Fitness & Instruction, Heal...",aD5Riwr4yPw4F9Xew2su2g,5,Nicole Anne Yoga is beyond amazing!! My daught...,2019-11-13 03:59:53,539


In [15]:
df.to_csv("yelp_reviews_Fitness_categories.csv", index=False)